In [ ]:
%matplotlib inline
%connect_info
%load_ext Cython
%load_ext autoreload
%autoreload 2  # 0 disable, 1 for only %aimport objects, 2 for those EXCEPT %aimport

# numpy and matplotlib already imported
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import fitsio
import pandas as pd

import ipywidgets
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
py.sign_in("cpadavis", "4oa3kd6g16")

In [ ]:
expid = 180489
kils = False
if kils:
    # these give the deconvolved stars
    deconv_dir = '/afs/slac.stanford.edu/u/ki/swmclau2/Git/DeconvolvePSF/DeconvolvePSF/output/00180489'
    # not sure what stars these really are? the combined psfex + deconv?
    deconvmodel_loc = '/nfs/slac/g/ki/ki18/des/cpd/180489_stars.npy'
    jamierod_results_path = '/nfs/slac/g/ki/ki18/des/cpd/jamierod_results.csv'
    mesh_directory = '/nfs/slac/g/ki/ki22/roodman/ComboMeshesv20'
    # directory containing the input data files
    base_directory = '/nfs/slac/g/ki/ki18/des/cpd/psfex_catalogs/SVA1_FINALCUT/psfcat/'
else:
    # these give the deconvolved stars
    deconv_dir = '/Users/cpd/Desktop/deconvpsf/00180000/00180489/'
    # not sure what stars these really are? the combined psfex + deconv?
    deconvmodel_loc = '/Users/cpd/Desktop/deconvpsf/00180000/00180489/180489_stars.npy'
    jamierod_results_path = '/Users/cpd/Desktop/deconvpsf/jamierod_results.csv'
    mesh_directory = '/Users/cpd/Projects/WavefrontPSF/meshes/Science-20121120s1-v20i2'
    # directory containing the input data files
    base_directory = '/Users/cpd/Desktop/deconvpsf'

# skeleton for making psf model

# this gets you a bunch of stamps

In [ ]:
from WavefrontPSF.psf_interpolator import Mesh_Interpolator
from WavefrontPSF.wavefront import Wavefront
from WavefrontPSF.digestor import Digestor
from WavefrontPSF.psf_evaluator import Moment_Evaluator
from WavefrontPSF.donutengine import DECAM_Model_Wavefront
from glob import glob
from itertools import izip
from psfex import PSFEx

jamierod_results = pd.read_csv(jamierod_results_path)
jamierod_results = jamierod_results.set_index('expid')

# set up objects. make sure I get the right mesh
digestor = Digestor()
PSF_Evaluator = Moment_Evaluator()
mesh_name = 'Science-20121120s1-v20i2_All'
PSF_Interpolator = Mesh_Interpolator(mesh_name=mesh_name, directory=mesh_directory)

# This will be our main wavefront
WF = DECAM_Model_Wavefront(PSF_Interpolator=PSF_Interpolator)

# load up data
expid_path = '/{0:08d}/{1:08d}'.format(expid - expid % 1000, expid)
data_directory = base_directory + expid_path
files = glob(data_directory + '/*{0}'.format('_selpsfcat.fits'))

data_df = digestor.digest_fits(files[0], do_exclude=False)
meta_hdulist = [fits.open(files[0])] #list of HDULists #META

for file in files[1:]:
    tmpData = digestor.digest_fits(file,do_exclude=False )
    data_df = data_df.append(tmpData)
    meta_hdulist.append(fits.open(file))

fit_i = jamierod_results.loc[expid]

misalignment = {'z04d': fit_i['z04d'], 'z04x': fit_i['z04x'], 'z04y': fit_i['z04y'],
                'z05d': fit_i['z05d'], 'z05x': fit_i['z05x'], 'z05y': fit_i['z05y'],
                'z06d': fit_i['z06d'], 'z06x': fit_i['z06x'], 'z06y': fit_i['z06y'],
                'z07d': fit_i['z07d'], 'z07x': fit_i['z07x'], 'z07y': fit_i['z07y'],
                'z08d': fit_i['z08d'], 'z08x': fit_i['z08x'], 'z08y': fit_i['z08y'],
                'z09d': fit_i['z09d'], 'z09x': fit_i['z09x'], 'z09y': fit_i['z09y'],
                'z10d': fit_i['z10d'], 'z10x': fit_i['z10x'], 'z10y': fit_i['z10y'],
                'rzero': fit_i['rzero']}

#print(misalignment['rzero'])
#rzero needs to be adjusted to be smaller than the stars!
x = .3 / .14#4-
misalignment['rzero'] = 1 / (1 / misalignment['rzero'] - x)
#print(misalignment['rzero'])

#print(.14*x )

data_df['rzero'] = misalignment['rzero']
optpsf, data_df = WF.draw_psf(data_df, misalignment=misalignment)

#optPSFStamps is a numpy data cube
#full_data is data frame including vignettes of the stars

# make the psfex models for both portions
psf_files = glob(data_directory + '/*{0}'.format('psfcat_validation_subtracted.psf'))
psfdeconv_files = glob(deconv_dir + '/*.psf')

atmpsf_list = []
psfex_list = []
stars = []
#TODO Check that files are in the same order as the hdulist
for psfex_file, file, hdulist in  izip(psf_files, psfdeconv_files, meta_hdulist):
    pex = PSFEx(file)
    pex_orig = PSFEx(psfex_file)
    for yimage, ximage in izip(hdulist[2].data['YWIN_IMAGE'], hdulist[2].data['XWIN_IMAGE']):
        atmpsf = np.zeros((32,32))
        #psfex has a tendency to return images of weird and varying sizes
        #This scheme ensures that they will all be the same 32x32 by zero padding
        #assumes the images are square and smaller than 32x32
        #Proof god is real and hates observational astronomers.
        atmpsf_small = pex.get_rec(yimage, ximage)
        atm_shape = atmpsf_small.shape[0] #assumed to be square
        pad_amount = int((32-atmpsf_small.shape[0]) / 2)
        atmpsf[pad_amount:32-(pad_amount+atm_shape%2),pad_amount:32-(pad_amount+atm_shape%2) ] = atmpsf_small
        atmpsf_list.append(atmpsf)
        
        atmpsf = np.zeros((32,32))
        #psfex has a tendency to return images of weird and varying sizes
        #This scheme ensures that they will all be the same 32x32 by zero padding
        #assumes the images are square and smaller than 32x32
        #Proof god is real and hates observational astronomers.
        atmpsf_small = pex_orig.get_rec(yimage, ximage)
        atm_shape = atmpsf_small.shape[0] #assumed to be square
        pad_amount = int((32-atmpsf_small.shape[0]) / 2)
        atmpsf[pad_amount:32-(pad_amount+atm_shape%2),pad_amount:32-(pad_amount+atm_shape%2) ] = atmpsf_small
        psfex_list.append(atmpsf)
    stars.append(hdulist[2].data['VIGNET'])

atmpsf = np.array(atmpsf_list)
psfexpsf = np.array(psfex_list)
stars = np.array(stars)
stars = np.vstack(stars).astype(np.float64)

model = np.load(deconvmodel_loc)

In [ ]:
def evaluate_stamps_and_combine_with_data(stamps, data):
    eval_data = WF.evaluate_psf(stamps)
    eval_data.index = data.index
    combined_df = eval_data.combine_first(data)
    return combined_df

In [ ]:
atmpsf_df = evaluate_stamps_and_combine_with_data(atmpsf, data_df)
psfexpsf_df = evaluate_stamps_and_combine_with_data(psfexpsf, data_df)
optpsf_df = evaluate_stamps_and_combine_with_data(optpsf, data_df)
model_df = evaluate_stamps_and_combine_with_data(model, data_df)
stars_df = evaluate_stamps_and_combine_with_data(stars, data_df)

In [ ]:
combinekeys = ['e0', 'e1', 'e2', 'E1norm', 'E2norm', 'delta1', 'delta2', 'zeta1', 'zeta2']
# make a big df with all the above columns combined
df = stars_df.copy()
names = ['model', 'psfex', 'opt', 'atm']
df_list = [model_df, psfexpsf_df, optpsf_df, atmpsf_df]
for key in combinekeys:
    # add the other medsub
    df['{0}_medsub'.format(key)] = df[key] - df[key].median()
    for name, psf in zip(names, df_list):
        if key == 'E1norm':
            psf[key] = psf['e1'] / psf['e0']
        elif key == 'E2norm':
            psf[key] = psf['e2'] / psf['e0']
        df['{0}_{1}'.format(name, key)] = psf[key]
        # add medsub
        df['{0}_{1}_medsub'.format(name, key)] = df['{0}_{1}'.format(name, key)] - df['{0}_{1}'.format(name, key)].median()
        df['{0}_{1}_diff'.format(name, key)] = df['{0}_{1}'.format(name, key)] - df[key]
        df['{0}_{1}_medsub_diff'.format(name, key)] = df['{0}_{1}_medsub'.format(name, key)] - df['{0}_medsub'.format(key)]

In [ ]:
# can play with this value to make the binning more or less fine
WF = Wavefront(model=df, num_bins=2)

# plot the wavefronts in terms of the moments for the different models

In [ ]:
def plot_wavefront(key):
    
    # set a and b vmin and vmax
    cutoff_percent = 5
    mins = [np.percentile(WF.field[key], cutoff_percent)] + \
           [np.percentile(WF.field['{0}_{1}'.format(name, key)], cutoff_percent) for name in names]
    maxs = [np.percentile(WF.field[key], 100 - cutoff_percent)] + \
           [np.percentile(WF.field['{0}_{1}'.format(name, key)], 100 - cutoff_percent) for name in names]
    a = min(mins)
    b = max(maxs)
    
    nrows = 3
    ncols = 2
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10 * ncols, 5 * nrows))
    ax = WF.prep_axis(axs.flatten()[0])
    WF.plot_field(key, fig=fig, ax=ax, a=a, b=b)
    ax.set_title('stars')
    for name_i, name in enumerate(names, start=1):
        ax = WF.prep_axis(axs.flatten()[name_i])
        WF.plot_field('{0}_{1}'.format(name, key), fig=fig, ax=ax, a=a, b=b)
        ax.set_title(name)
        
    # have an extra!
    axs.flatten()[-1].set_axis_off()
ipywidgets.interact(plot_wavefront, key=combinekeys + [key + '_medsub' for key in combinekeys])

# plot wavefront - data, psfex - data

In [ ]:
def plot_wavefront_diff(key):
    
    # set a and b vmin and vmax
    cutoff_percent = 5
    mins = [np.percentile(WF.field['{0}_{1}_diff'.format(name, key)], cutoff_percent) for name in names]
    maxs = [np.percentile(WF.field['{0}_{1}_diff'.format(name, key)], 100 - cutoff_percent) for name in names]
    a = min(mins)
    b = max(maxs)
    
    nrows = 2
    ncols = 1
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20 * ncols, 10 * nrows))
    for name_i, name in enumerate(['model', 'psfex']):
        ax = WF.prep_axis(axs.flatten()[name_i])
        WF.plot_field('{0}_{1}_diff'.format(name, key), fig=fig, ax=ax, a=a, b=b)
        ax.set_title(name)
ipywidgets.interact(plot_wavefront_diff, key=combinekeys + [key + '_medsub' for key in combinekeys])

In [ ]:
def view_stars(indx):
    nrows = 2
    ncols = 3
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(8 * ncols, 6 * nrows))
    names = ['star', 'model', 'psfex', 'opt', 'atm']
    stamps = [stars, model, psfexpsf, optpsf, atmpsf]
    for ith, name, stamp in zip(range(len(names)), names, stamps):
        ax = axs.flatten()[ith]
        star = stamp[indx].copy()
        star = np.where(star < -1e10, 0, star)
        star /= star.sum()
#         star = np.log10(star)
        IM = ax.imshow(star, cmap=plt.cm.RdBu_r)
        fig.colorbar(IM, ax=ax)
        ax.set_title('{0} {1}'.format(name, indx))
    # have an extra!
    axs.flatten()[-1].set_axis_off()
ipywidgets.interact(view_stars, indx=(0, len(stars), 1))